In [1]:
from collections import OrderedDict, deque
import numpy as np
import itertools
import networkx as nx
from networkx.drawing.nx_pydot import write_dot
from networkx.drawing.nx_pydot import pydot_layout
import matplotlib as mpl
# mpl.use('pdf')
import matplotlib.pylab as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox, TextArea, DrawingArea
from matplotlib.patches import Circle
from numba import jit
# import pickle
import cPickle as pickle
import gzip

import locale
locale.setlocale(locale.LC_ALL, 'en_US.utf8')

import os
import psutil
process = psutil.Process(os.getpid())

In [2]:
mpl.rcParams['figure.dpi'] = 1600
mpl.rcParams['figure.figsize'] = (
    20,
    20
    )
mpl.rcParams['savefig.transparent'] = True

In [3]:
# Import Images
Images = dict()
from glob import glob
for item in glob("./AllPatterns/*.png"):
    try:
        Filename = item.split("/")[2]
        print Filename
        Images[int(Filename[:2])] = Filename
    except:
        print Filename
        raise SystemExit

52  1 1 0.1 0 0.png
51  1 1 0.0 1 1.png
56  1 1 1.0 0 0.png
53  1 1 0.1 0 1.png
54  1 1 0.1 1 0.png
50  1 1 0.0 1 0.png
32  1 0 0.0 0 0.png
48  1 1 0.0 0 0.png
36  1 0 0.1 0 0.png
63  1 1 1.1 1 1.png
60  1 1 1.1 0 0.png
62  1 1 1.1 1 0.png


In [4]:
import inspect
def PrintIter(ob):
    frame = inspect.currentframe()
    frame = inspect.getouterframes(frame)[1]
    string = inspect.getframeinfo(frame[0]).code_context[0].strip()
    arg = string[string.find('(') + 1:-1].split(',')[0]
#     print arg
#     print "{} : {}".format([ k for k,v in locals().iteritems() if (v == ob and not k == 'ob')][0], type(ob))
    print "{} : {}\n".format(arg, type(ob))
    if type(ob) == OrderedDict or type(ob) == dict:
        for x, y in ob.items():
            print "{: >2} : {}".format(x, y)
    elif (type(ob) == list or type(ob) == np.ndarray) and not arg=="D.Reactions":
        for x, y in enumerate(ob):
            print "{: >2} : {}".format(x, y)
    elif arg=="D.Reactions":
#         print "Converted to Species Int:"
        for x, y in enumerate(ob):
            print "{: >3} : {: >2}, {: >2} -> {: >2} : {}".format(x, D.ReactionRelations(y)[0],D.ReactionRelations(y)[1],D.ReactionRelations(y)[2], y)
    else:
        print ob


In [5]:
def SpeciesToRepString(Species, N=3):
    if isinstance(Species, np.ndarray):
        Out = map(str, Species.flatten())
        Out = ",".join(Out[:N]) + ";" + ",".join(Out[N:])
        return Out
    if isinstance(Species, int):
        Out = [x for x in str(bin(Species)[2:].zfill(N*2))]
        Out = ",".join(Out[:N]) + ";" + ",".join(Out[N:])
        return Out
    if isinstance(Species, str) and len(Species) > 4*N-1:
        return Species[:N*4-1]
    elif len(Species) == N*4-1:
        return Species
#     Cur = Species.split(" ")
#     if len(Cur) == 2 and len(Cur[0]) == 4*N-1:
#         return Cur[0]
#     if Cur[0] == Species:
#         return Species
    return False

def SpeciesToArrays(Species, N=3):
    Cur = SpeciesToRepString(Species, N)
    Cur = np.array([x.split(",") for x in Cur.split(";")], dtype="b")
    return Cur

def SpeciesArrayToInt(Species):
    Species = SpeciesToArrays(Species)
    return int(np.sum([x*(2**n) for n,x in enumerate(Species.flatten()[::-1])]))

def Symmetries(Species, N=3, Unique = True, Ints=False, Sorted=True):
    '''
    Symmetries(Species, N=3, Unique = True, Ints=False, Sorted=True)
    Sorted = False results in the index consistantly representing the same symmetry
    Results are sorted, so first entry is always the "Cannonical Rep"
    '''
    Cur = SpeciesToArrays(SpeciesToRepString(Species, N), N)
    Syms = []
    Syms.append(Cur) # Identity
    for I in range(1,N): # Rotational Symmetries (C_N)
        Syms.append(np.roll(Cur, I, axis=1))
    for I in range(len(Syms)): # Reflection Symmetries
        Syms.append(Syms[I][::-1])
#     for S in Syms:
#         print("{} : {} : {}".format(SpeciesArrayToInt(S),SpeciesToRepString(S),S))
    if Unique:
#         print set(map(SpeciesArrayToInt,Syms))
        Out = map(SpeciesToArrays,set(map(SpeciesArrayToInt,Syms)))
    else:
        Out = Syms
    if Sorted:
        Out = sorted(Out, key=SpeciesArrayToInt)
    else:
        pass
    Out = [SpeciesToRepString(x, N) for x in Out]
    if Ints:
        return map(SpeciesArrayToInt,Out[::-1])
    else:
        return [SpeciesToRepString(x, N) for x in Out[::-1]]


In [6]:
# def SpeciesToIndex(S, Species):
#     return int(np.where(np.array(Species.values())==S)[0])
def SpeciesToIndex(S, Species = False):
    SpeciesIndex = {y:i for (i, (x,y)) in enumerate(Species.items())}
    return SpeciesIndex[S]

# def SpeciesToInt(S):
#     return int(S[::2],2)
def SpeciesToName(S, Species):
    return Species.values()[S]

class Probabilities(object):
    def __init__(self, Data, Concentrations=False):
        self.Data = Data
        self.ProteinsInSpecies = [sum(x == "1" for x in Data.Species.values()[i]) for i in range(Data.SpeciesCount)]
        self._GenReactionsByProduct()
        self.UpdateNorms(Concentrations)

    def LookupReaction(self, Product, R1 = -1, R2 = -1):
        Out = deque()
        for R in self.Data.Reactions:
            if R[3] == Product:
                Out.append(R)
        if R1 == -1 and R2 == -1:
            return Out
        R1 = int(R1)
        R2 = int(R2)
        for R in Out:
            if R[0] == R1 and R[1] == R2:
                return R
            if R[1] == R1 and R[0] == R2:
                return R
        return False

    def _GenReactionsByProduct(self):
        ReactionsByProduct = OrderedDict()
        for Species in self.Data.SpeciesConvert.values():
            for R in self.LookupReaction(Species):
                try:
                    ReactionsByProduct[Species].append(R)
                except:
                    ReactionsByProduct[Species] = list()
                    ReactionsByProduct[Species].append(R)
        self.ReactionsByProduct = ReactionsByProduct

    def UpdateNorms(self, Concentrations = False): 
        # Generate the normalization factors (Return list in the same order as the Data.Species list)
        if hasattr(Concentrations, '__iter__'):
            NormsBySpecies = np.zeros(len(Concentrations))
        else:
            Concentrations = np.ones(self.Data.SpeciesCount)
            NormsBySpecies = np.zeros(self.Data.SpeciesCount)

        for i, Rs in self.ReactionsByProduct.items():
            #print "{} : {}".format(i,Rs)
            for R in Rs:
                # R[2]: Forward
                # R[0]: Reactant 1
                # R[1]: Reactant 2
                # R[3]: Product
                NormsBySpecies[i] += R[2]*Concentrations[R[0]]*Concentrations[R[1]]
                #print "{} : {}".format(R, NormsBySpecies[i])
        self.Concentrations = Concentrations 
        self.NormsBySpecies = NormsBySpecies

    def UpdateConcentrations(self, Fraction, Concentration): 
        #RealConc = np.zeros_like(Fraction)
        RealConc = Fraction*Concentration/self.ProteinsInSpecies
        self.UpdateNorms(RealConc)

    def ReactionFlux(self, Product, R1, R2):
        Forward = self.LookupReaction(Product, R1, R2)
        if hasattr(Forward, '__iter__'):
            Forward = Forward[2]
        else:
            print "Reaction for P: {} R1: {} R2: {} Not Found!!!!".format(Product, R1, R2)
        if self.Concentrations[R1]*self.Concentrations[R2] != 0.0:
            return Forward*self.Concentrations[R1]*self.Concentrations[R2]/(self.NormsBySpecies[Product])
        return 0.0

    def NodeFlux(self, Graph, Node = "Root"):
        if Node == "Root":
            Node = self.Data.Species.values()[-1] # Root Node Name
        Product = nx.get_node_attributes(Graph,"Product")[Node]
        R1 = nx.get_node_attributes(Graph,"Reactant1")[Node]
        R2 = nx.get_node_attributes(Graph,"Reactant2")[Node]
        return self.ReactionFlux(Product,R1,R2)

    def PathIndependentFlux(self, Graph, Probability = True):
        Product = nx.get_node_attributes(Graph,"Product").values()
        R1 = nx.get_node_attributes(Graph,"Reactant1").values()
        R2 = nx.get_node_attributes(Graph,"Reactant2").values()
        Flux = 1.0
        for i in range(len(Product)):
            Flux *= self.ReactionFlux(Product[i],R1[i],R2[i])
        if not Probability:
            # Remove normalization from last step
            Flux *= self.NormsBySpecies[-1]
        return Flux

    def GetChildren(self, Graph, Node):
        return Graph.successors(Node)


In [7]:
import copy
class Pathway(object):
    def __init__(self):
        self.childrendict = dict()
        self.degreedict = dict()
    
    def degree(self):
        return self.degreedict
    
    def nodes(self, data=False):
        return tuple(self.degreedict.keys())

    def add_node(self, nodename):
        if len(self.nodes()) == 0:
            self.degreedict[nodename] = 1
        else:
            self.degreedict[nodename] = 0
    
    def children(self, nodename):
        try:
            return self.childrendict[nodename]
        except:
            return ()
    
    def add_edge(self, node1, node2):
        try:
            self.degreedict[node2] += 1
        except:
            self.degreedict[node2] = 1
        try:
            self.childrendict[node1].append(node2)
            self.degreedict[node1] += 1
        except:
            self.degreedict[node1] = 2
            self.childrendict[node1]=list([node2])
            
    def edges(self):
        Edge = []
        for key, value in self.childrendict.items():
            for v in value:
                Edge.append((key, v))
        return tuple(Edge)
        
    def copy(self):
        New = Pathway()
        New.childrendict = copy.copy(self.childrendict)
        New.degreedict = copy.copy(self.degreedict)
        return New

#     def copy(self):
#         New = Pathway()
#         New.childrendict = copy.deepcopy(self.childrendict)
#         New.degreedict = copy.deepcopy(self.degreedict)
#         return New
    

In [8]:
class ImportedData(object):
    def __init__(self, N = 3):
        self.N = N
        self.SymCount = 2*N
        self.GetSDataFromFiles("species{}".format(N))
        self.GetRDataFromFiles("react{}".format(N))
        #         Sort Species
        self.MonomerCount = int(len(self.Species.values()[0])/2)
        self.RootNodeName = SpeciesToRepString(2**(2*N)-1, N)
        self.MonomerName  = SpeciesToRepString(2**(2*N-1), N)
        self.SpeciesIndex = {y:i for (i, (x,y)) in enumerate(self.Species.items())}
        self.CreateReactionLookup()
        self.SpeciesRepList = self.Species.values()
        
    def CreateReactionLookup(self):
        Reactions = np.array(self.Reactions)
        self.ReactionLookupBySpecies = dict()
        for i, Key in enumerate(map(frozenset, Reactions[:,(0,1,3)])):
            self.ReactionLookupBySpecies[Key] = i
    
    def GetSDataFromFiles(self, filename):
        try:
            self.Species = OrderedDict()
            with open(filename,"r") as SpeciesFile:
                SpeciesList = SpeciesFile.read().strip().split("\n")
                for S in SpeciesList:
                    L = S.split(";;")
                    self.Species[int(L[0])] = L[1]
        
            self.Species = OrderedDict(sorted(self.Species.items(), key=lambda x: x[0]))
            self.SpeciesConvert = OrderedDict()
            for i, S in enumerate(map(int,self.Species.keys())):
                self.SpeciesConvert[int(S)] = i

            self.SpeciesCount = len(self.Species.keys())
            self.SpeciesNumbers = np.unique(np.array([Symmetries(x, Ints=True, Unique=False) for x in self.Species.keys()[::-1]]).flatten())
            
            
        except:
            return False

    def GetRDataFromFiles(self, filename):
        try:
            self.Reactions = []
            with open(filename,"r") as ReactFile:
                ReactList = ReactFile.read().strip().split("\n")
                ReactList = map(lambda x: x.split("\t"), ReactList)
                for R in ReactList:
                    R = map(lambda x: x.split(";;"), R)
                    R = map(lambda x: x[0], R)
                    R = map(lambda x: x.split(","), R)
                    # Drop Representation
                    R = list(itertools.chain(*R))
                    R = map(int, R)
                    R[0]=self.SpeciesConvert[R[0]]
                    R[1]=self.SpeciesConvert[R[1]]
                    if R[0] > R[1]:
                        R0 = R[0]
                        R1 = R[1]
                        R[0] = R1
                        R[1] = R0
                    R[3]=self.SpeciesConvert[R[3]]
                    self.Reactions.append(R)

                # Sort Reactions by Product, R1, R2 (R1 is always less than R2)
                SortList = map(list,list(np.array(self.Reactions)[:,(3,0,1)]))
                L = range(len(SortList))
                for i in L:
                    SortList[i].append(i)
                SortOrder = np.array(sorted(SortList))[:,3]
                self.Reactions = map(list,list(np.array(self.Reactions)[SortOrder]))
                
                return True
        except Exception as e:
            print e
            return False
        
    def ReactionValue(self, Node, Graph): # Used for cannonical definition
        def AddToNode(N):
            if N >= 0:                
                ##  0  1 2 3  4  5 6
                ## R1 R2 F P B1 B2 R
                Graph.add_node(Node, {
                                            "RIndex":N,
                                            "Forward":self.Reactions[N][2],
                                            "Reverse":self.Reactions[N][6],
                                            "BondType1":self.Reactions[N][4],
                                            "BondType2":self.Reactions[N][5],
                                            "Product":self.Reactions[N][3],
                                            "Reactant1":self.Reactions[N][0],
                                            "Reactant2":self.Reactions[N][1],
                                            })
            else:
                Graph.add_node(Node, RIndex = -1)
#         P = SpeciesToIndex(Node.split(" ")[0], self.Species)
        P = self.SpeciesIndex[Node[:self.N*4-1]]
#         if P == min(self.SpeciesConvert.values()):
        if P == 0:
            AddToNode(-1)
            return 0
        #if P == min(self.SpeciesConvert.values()) or P == max(self.SpeciesConvert.values()):
        #    AddToNode(0)
        #    return 0
        R1, R2 = Graph.succ[Node].keys()

        R1 = self.SpeciesIndex[R1[:self.N*4-1]]
        R2 = self.SpeciesIndex[R2[:self.N*4-1]]
#         R1 = SpeciesToIndex(R1.split(" ")[0], self.Species)
#         R2 = SpeciesToIndex(R2.split(" ")[0], self.Species)
        i = self.ReactionLookupBySpecies[frozenset([P, R1, R2])]
        AddToNode(i)
        return i+1
#         for i, R in enumerate(self.Reactions):
#             if P == R[3]:
#                 if R1 in R[:2] and R2 in R[:2]:
#                     AddToNode(i)
#                     return i+1
#         else:
#             raise KeyError
            
    def SpeciesNumberLookup(self, SN):
        return np.argmax(self.SpeciesNumbers==SN)
    
    def ReactionRelations(self, Reaction):
        '''
        def ReactionRelations(self, Reaction)
        Returned the Int reps of the Product and Reactants fo the Reaction Array Sumbitted.
        '''
        return (self.Species.keys()[Reaction[3]], self.Species.keys()[Reaction[0]], self.Species.keys()[Reaction[1]])
    
    def FindReactions(self, Reaction):
        PRep = SpeciesToRepString(Parent, self.N)
        
    
    def KnownStructureCheck(self, Entry):
        if np.sum(self.SpeciesNumbers == SpeciesArrayToInt(Entry)) > 0:
            return True
        else:
            return False

    def Children(self, Parent, Child):
        '''
        Parent : int
        Child1 : int
        Child2 : int
        '''
        PAS = np.array(Symmetries(Parent, Unique=False, Ints=True, Sorted=True))
        C1S = np.array(Symmetries(Child, Unique=False, Ints=True, Sorted=True))
        PI  = np.argmax(PAS == Parent)
        PC  = PAS[PI]
        I = 0
        while True:
            C1C = C1S[PI + I]
            PA = SpeciesToArrays(PC)
            C1 = SpeciesToArrays(C1C)
            C2 = PA - C1
            if I > 7:
                return False
            if self.KnownStructureCheck(SpeciesArrayToInt(C2)):
                break
            I += 1
#         print PA
#         print C1
#         print C2
    #     if (PA != C1+C2).all():
    #         return False
    #     if np.sum(PA) != (np.sum(C1) + np.sum(C1)):
    #         return False
        POut = SpeciesArrayToInt(PA)
        C1Out = SpeciesArrayToInt(C1)
        C2Out = SpeciesArrayToInt(C2)
        if self.SpeciesNumberLookup(C1Out) > self.SpeciesNumberLookup(C2Out):
            pass
        else:
            CT = C1Out
            C1Out = C2Out
            C2Out = CT
    #     return self.SpeciesNumberLookup(POut), self.SpeciesNumberLookup(C1Out), self.SpeciesNumberLookup(C2Out)
        return POut, C1Out, C2Out
    #     return SpeciesToArrays(POut), SpeciesToArrays(C1Out), SpeciesToArrays(C2Out)
    
# def AddReactionDataToGraph(Graph, Data):
#     map(lambda i: Data.ReactionValue(i, Graph), Graph.nodes())
#     return list()

def LeftRight(N1, N2, Graph, Data):
    R1 = Graph.node[N1]["RIndex"]
    R2 = Graph.node[N2]["RIndex"]
    if R1 > R2:
        return N1, N2
    elif R1 < R2:
        return N2, N1
    elif R1 == R2:
        R1 = CannonicalGraph(Graph, Data, Start = N1).values()
        R2 = CannonicalGraph(Graph, Data, Start = N2).values()
        for i in range(max(len(R1), len(R2))):
            try:
                if R1[i] > R2[i]:
                    return N1, N2
                if R1[i] < R2[i]:
                    return N2, N1
            except Exception as e:
                print e
                if len(R1) > len(R2):
                    return N1, N2
                elif len(R2) > len(R1):
                    return N2, N1
            return N1, N2

# def CannonicalGraph(Graph, Data, Start):
#     map(lambda i: Data.ReactionValue(i, Graph), Graph.nodes())
#     return frozenset(Graph.nodes())

def CannonicalGraph(Graph, Data, Start):
    map(lambda i: Data.ReactionValue(i, Graph), Graph.nodes())
#     AddReactionDataToGraph(Graph, Data)
    CForm = OrderedDict()
    Todo = deque()
    Todo.append(Start)
    while len(Todo) > 0:
        C = Todo.popleft()
        CForm[C] = Graph.node[C]["RIndex"]
        Children = Graph.succ[C].keys()
        if len(Children) != 2:
            pass
        else:
            C1, C2 = Children
            R1 = Graph.node[C1]["RIndex"]
            R2 = Graph.node[C2]["RIndex"]
            I1, I2 = LeftRight(C1, C2, Graph, Data)
            Todo.appendleft(I1)
            Todo.append(I2)
    return CForm

Max = 1
def UniqueNodeName(Graph, NodeName):
    global Max
    Max += 1
    return "{} {}".format(NodeName,Max)

import itertools

def NodeConvert(NodeName, Data):
    IntValue = Data.SpeciesIndex[NodeName[:Data.N*4-1]]
    return (IntValue, NodeName)

def ChildrenValues(Node, Graph, Data):
#     Children = Graph.succ[Node].keys()
    Children = Graph.children(Node)
    return sorted([(Data.SpeciesIndex[NodeName[:Data.N*4-1]], NodeName) for NodeName in Children if Data.SpeciesIndex[NodeName[:Data.N*4-1]] != 0])

def CannonicalIndices(Graph, Data, Start):
    # Not Needed for Pathway calculations, but is needed for flux calculations
#     map(lambda i: Data.ReactionValue(i, Graph), Graph.nodes())
    
    CForms = set()
    StartNodeInt = NodeConvert(Start, Data)[0]
    
    BranchingNumber = 0
    ChildrenLookup = dict()
    BranchingChildren = dict()
    for Node in Graph.nodes():
        ChildrenLookup[Node] = ChildrenValues(Node, Graph, Data)
        if len(ChildrenLookup[Node]) == 2:
            BranchingChildren[Node] = ChildrenLookup[Node]
            BranchingNumber += 1
    BranchingNumber = len(BranchingChildren.keys())
    
    # Left Right Branching True = Left False = Right
    Combinations = [x for x in itertools.combinations_with_replacement((True,False),BranchingNumber)]
#     PrintIter(BranchingChildren)
#     PrintIter(Combinations)
    for C in Combinations:
        C = deque(C)
        Todo = deque()
        Todo.append(Start)
        CForm = []
        while len(Todo) > 0:
            CNode = Todo.pop()
            Children = ChildrenLookup[CNode]
            if len(Children) == 2:
                Branch = C.pop()
                if Branch:
                    CForm.append(Children[0][0])
                    Todo.append(Children[0][1])
                    Todo.append(Children[1][1])
                else:
                    CForm.append(Children[1][0])
                    Todo.append(Children[1][1])
                    Todo.append(Children[0][1])
            elif len(Children) == 1:
                    CForm.append(Children[0][0])
                    Todo.append(Children[0][1])
#                     CForm.append(-1)
            else:
#                 CForm.append(-1)
#                 CForm.append(-1)
                pass
        CForms.add(tuple(CForm))
#     print "----------------------"
#     print BranchingNumber
#     PrintIter(ChildrenLookup)
#     return sorted(CForms)[::-1]
    return tuple([x for sublist in sorted(CForms,reverse=True) for x in sublist])

In [9]:
def ImageLookup(NodeName):
    """
    :string Nodename:
    :return string Filename:
    """
    ID = np.sum(np.reshape(np.array(map(lambda x: np.array(x.split(","), dtype=int), NodeName.split(" ")[0].split(";"))), 6) * [32, 16, 8, 4, 2, 1])

    return Images[ID]

def imscatter(x, y, image, ax=None, zoom=1, text=False):
    if ax is None:
        ax = plt.gca()
    if not text:
        try:
            image = plt.imread(image)
        except TypeError:
            # Likely already an array...
            pass
    if text:
        im = TextArea(image, minimumdescent=False, textprops=dict(size='large',weight='bold'))
        da = DrawingArea(20, 20, 0, 0)
        p = Circle((10, 10), 50, color="red")
        da.add_artist(p)
    else:
        im = OffsetImage(image, zoom=zoom)
    x, y = np.atleast_1d(x, y)
    artists = []
    for x0, y0 in zip(x, y):
        if text:
#             ab1 = AnnotationBbox(da, im, (x0, y0), xycoords='data', frameon=False)
            ab1 = AnnotationBbox(da, (x0, y0), xycoords='data', frameon=False)
            ab2 = AnnotationBbox(im, (x0, y0+9), xycoords='data', frameon=False)
#             artists.append(ax.add_artist(ab1))
            artists.append(ax.add_artist(ab1))
            artists.append(ax.add_artist(ab2))
        else:
            ab = AnnotationBbox(im, (x0, y0), xycoords='data', frameon=False)
            artists.append(ax.add_artist(ab))
    ax.update_datalim(np.column_stack([x, y]))
    ax.autoscale()
    return artists

def Plot(Graph, with_labels=True, Show=False, pretty_images=False):
    Fig = plt.figure()
    Axes = Fig.add_subplot(111)
    write_dot(Graph,'test.dot')
    #Fig.set_title("Tree Plot")
    pos=pydot_layout(Graph, prog="dot")
    # nx.draw(Graph,pos,with_labels=with_labels,arrows=False, ax=Axes, hold=True)
    nx.draw(Graph, pos, with_labels=False, arrows=False, ax=Axes, hold=True, nodeshape = "o", width=10.0)

    #ZoomSize = 1.8*10**-0
    ZoomSize = 0.2
    if pretty_images == "text":
        for NodeName, Loc in pos.items():
            imscatter(Loc[0], Loc[1], "\n"*2+NodeName.split(" ")[0].replace(";","\n"), zoom=ZoomSize, text=True)
    elif pretty_images:
        for NodeName, Loc in pos.items():
            imscatter(Loc[0], Loc[1], "{}{}".format('./AllPatterns/', ImageLookup(NodeName)), zoom=ZoomSize)
    # Axes.plot(plot_data)
    for T in Axes.texts:    # Remove unique node labels.
        if T._text.split(" ")[0] == "1,0,0;0,0,0":
            T._text = "1"
        else:
            T._text = "\n"*4+T._text.split(" ")[0].replace(";","\n")
    if Show:
        plt.show()
    return Fig

In [10]:
import time
def pretty_time_delta(seconds):
    sign_string = '-' if seconds < 0 else ''
    seconds = abs(int(seconds))
    days, seconds = divmod(seconds, 86400)
    hours, seconds = divmod(seconds, 3600)
    minutes, seconds = divmod(seconds, 60)
    if days > 0:
        return '%s%dd%dh%dm%ds' % (sign_string, days, hours, minutes, seconds)
    elif hours > 0:
        return '%s%dh%dm%ds' % (sign_string, hours, minutes, seconds)
    elif minutes > 0:
        return '%s%dm%ds' % (sign_string, minutes, seconds)
    else:
        return '%s%ds' % (sign_string, seconds)

In [11]:
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [12]:
import math
def catalan(n):
    return int(math.factorial(2*n)/(math.factorial(n+1)*math.factorial(n)))

In [13]:
def PathwayFlux(SortedPathways, FluxFunc):
    Flux = deque()
    for Path in SortedPathways.values():
        # By multiplying by the final step normilazation the cancelation will weight the system by the final flux
        Flux.append(FluxFunc(Path))
    return Flux

# def Reactants(P, Reactions, Species):
#     PR = deque()
#     for R in Reactions:
#         PT = (SpeciesToIndex(P, Species) == R[3])
#         if PT:
#             PR.append((R[0], R[1], R[3]))
#     return PR
@jit
def Reactants(P, Reactions, Species):
    ReactionsArray = np.array(Reactions)
    PR = deque(map(tuple,ReactionsArray[SpeciesToIndex(P, Species) == ReactionsArray[:,3]][:,(0,1,3)]))
    return PR

# ReactionsArray = np.array(D.Reactions)
# deque(map(tuple,ReactionsArray[5 == ReactionsArray[:,3]][:,(0,1,3)]))

@jit
def BuildReactionsWithSpeciesInvolvedLookupTables(Data):
    LookupTable = dict()
    ReactionsArray = np.array(Data.Reactions)
    ##  0  1 2 3  4  5 6
    ## R1 R2 F P B1 B2 R
    for SpeciesRep, SpeciesIndex in Data.SpeciesIndex.items():
        LookupTable[SpeciesRep] = deque(map(tuple,ReactionsArray[SpeciesIndex == ReactionsArray[:,3]][:,(0,1,3)]))
    return LookupTable

def AsmPathways(N=3, Unique = False, Output = False, SaveFiles = False):
    process = psutil.Process(os.getpid())
    
    StartTime = time.time()
    ChunkTime = time.time()
    D = ImportedData(N)

    ReactantsLookup = BuildReactionsWithSpeciesInvolvedLookupTables(D)
    
    RootNodeName = D.RootNodeName
    MonomerName  = D.MonomerName
    Tasks = dict()
#     Tree = nx.DiGraph()
    Tree = Pathway()
    Tree.add_node(RootNodeName)
    Tasks[Tree] = deque([RootNodeName, ])
    SaveGraphs = []
    CompletedGraphs = 0
#     while np.sum(map(len, Tasks.values())) > 0:
#         for Key in [x for x, y in Tasks.items()]:
    PreviousStartingTree = Tree
    
    GraphGCs = []

    FileNumber = 1
    Current_Round_File = 1
    Current_Round_Status = 1
    Delta_Output_File   = int(5e5)
    Delta_Output_Status = Delta_Output_File
    FileList = []
    FileListGC = []
    while True:
        Key = PreviousStartingTree
        
        try:
            LastNode = Tasks[Key].pop()
        except:
            try:
                Key = next(iter(Tasks.iterkeys()))
#                 Key = Tasks.keys()[0]
            except:
                break
            LastNode = Tasks[Key].pop()
        
        G = Key

        Rs = ReactantsLookup[LastNode[:D.N*4-1]]

        if len(Rs) > 1:
            Tree = G.copy()
            PreviousDeque = copy.copy(Tasks[G])
        for i, R in enumerate(Rs):
            
            PreviousStartingTree = Key
            
            if i > 0:
                G = Tree.copy()
                Tasks[G] = copy.copy(PreviousDeque)

            SN1 = D.SpeciesRepList[R[0]]
            NodeName1 = UniqueNodeName(G, SN1)

            G.add_node(NodeName1)
            G.add_edge(LastNode, NodeName1)

            SN2 = D.SpeciesRepList[R[1]]
            NodeName2 = UniqueNodeName(G, SN2)

            G.add_node(NodeName2)
            G.add_edge(LastNode, NodeName2)

            if SN1 != MonomerName:
                Tasks[G].append(NodeName1)
            if SN2 != MonomerName:
                Tasks[G].append(NodeName2)
            
            if len(G.nodes()) == D.N*4-1:
                CompletedGraphs += 1
                del Tasks[G]
#                 if len(SaveGraphs) < 10e10:
                SaveGraphs.append(G)
                
                GraphGCs.append(CannonicalIndices(G, D, D.RootNodeName))
                
                Current_Round_Status += 1
                Current_Round_File   += 1
                if Output and Current_Round_Status > Delta_Output_Status:
                    print "Time Running: {: >10} | Chunk Time Delta: {: >10} | Graphs found so far: {: >15} | Current SaveGraphs Length: {: >15} | Current Memory Usage: {: >10}".format(
                        pretty_time_delta(time.time() - StartTime),
                        pretty_time_delta(time.time() - ChunkTime),
                        locale.format("%d",CompletedGraphs, grouping=True),
                        locale.format("%d", len(SaveGraphs), grouping=True),
                        sizeof_fmt(process.memory_info()[0])
                    )
                if SaveFiles and Current_Round_File > Delta_Output_File:
                    FileName = "FullPathways{}_Chunk{}.pickle".format(D.N, FileNumber)
                    FileWriteStartTime = time.time()
                    with gzip.open(FileName, "w") as fh:
                        pickle.dump(SaveGraphs[:Delta_Output_File], fh, pickle.HIGHEST_PROTOCOL)
                        print "File: {} written in {: >5}".format(FileName, pretty_time_delta(time.time() - FileWriteStartTime))
                        FileList.append(FileName)
                    FileName = "PathwayGCs{}_Chunk{}.pickle".format(D.N, FileNumber)
                    FileWriteStartTime = time.time()
                    del SaveGraphs[:Delta_Output_File]
                    with gzip.open(FileName, "w") as fh:
                        pickle.dump(GraphGCs[:Delta_Output_File], fh, pickle.HIGHEST_PROTOCOL)
                        print "File: {} written in {: >5}".format(FileName, pretty_time_delta(time.time() - FileWriteStartTime))
                        FileListGC.append(FileName)
                    del GraphGCs[:Delta_Output_File]
                    FileNumber += 1
                    Current_Round_File -= Delta_Output_File
                if Output and Current_Round_Status > Delta_Output_Status:
                    ChunkTime = time.time()
                    Current_Round_Status -= Delta_Output_Status
                
    if SaveFiles:
        FileName = "FullPathways{}_Chunk{}.pickle".format(D.N, FileNumber)
        FileWriteStartTime = time.time()
        with gzip.open(FileName, "w") as fh:
            pickle.dump(SaveGraphs, fh, pickle.HIGHEST_PROTOCOL)
            print "File: {} written in {: >5}".format(FileName, pretty_time_delta(time.time() - FileWriteStartTime))
            FileList.append(FileName)
        del SaveGraphs
        FileName = "PathwayGCs{}_Chunk{}.pickle".format(D.N, FileNumber)
        with gzip.open(FileName, "w") as fh:
            pickle.dump(GraphGCs, fh, pickle.HIGHEST_PROTOCOL)
            print "File: {} written in {: >5}".format(FileName, pretty_time_delta(time.time() - FileWriteStartTime))
            FileListGC.append(FileName)
        del GraphGCs

        FileNumber += 1
                    
#                     if D.SpeciesIndex[SN1] != 0:
#                         Tasks[G].append(NodeName1)
#                     if D.SpeciesIndex[SN2] != 0:
#                         Tasks[G].append(NodeName2)
#                 Degree = [x for x,y in G.degree().items() if y!=3 and x != D.RootNodeName and x[:D.N*4-1] != D.MonomerName and x not in Tasks[G]]
#                 for K in Degree:
#                     if Output:
#                         print "Degree Check Used!!! : {}".format(K)
#                     Tasks[G].append(K)

#             [Tasks[G].append(x) for x,y in G.degree().items() if y!=3 and x != D.RootNodeName and x[:D.N*4-1] != D.MonomerName and x not in Tasks[G]]

#             Degree = G.degree()
#             for K in Degree.keys():
#                 if K[:D.N*4-1] == RootNodeName:
#                     pass
#                 elif K[:D.N*4-1] == MonomerName:
#                     pass
#                 elif Degree[K] != 3 and K not in Tasks[G]:
#                     if Output:
#                         print "Degree Check Used!!! : {}".format(K)
#                     Tasks[G].append(K)
    
    if Output:
        print "-----------------------"+"-"*10
        print "Maximum Possible Trees {: >10}".format(catalan(D.N*2 - 1))
        print "Full Length Trees:     {: >10}".format(CompletedGraphs)
        if not SaveFiles:
            print "Total Trees Found:     {: >10}".format(len(SaveGraphs))
            print "Wrong Node Count:      {: >10}".format(len([x for x in SaveGraphs if len(x.nodes()) != D.N*4-1]))
            print "Wrong Degree Count:    {: >10}".format(len([x for x,y in G.degree().items() if y!=3 and x != D.RootNodeName and x[:D.N*4-1] != D.MonomerName for G in SaveGraphs]))
#         print "Wrong Degree Count:    {: >10}".format(len([x for x,y in G.degree().items() if y!=3 and x != D.RootNodeName and x[:D.N*4-1] != D.MonomerName for G in SaveGraphs]))

    if not SaveFiles:
        return D, SaveGraphs
    if SaveFiles:
        return D, FileList, FileListGC

def UniquifyGraphs(D, SaveGraphs, Output = False):
    # Unique Graphs
    Graphs = {}
    InvalidGraphs = {}
    InvalidGraphsCount = {}
    
#     def InList(L1, L2, Index = False):
#         N1 = np.array(L1)
#         for k, i in enumerate(L2):
#             Ni = np.array(i)
#             if (N1 == Ni).all():
#                 if Index:
#                     return True, k
#                 else:
#                     return True
#         return False
#     from numba import njit, prange
    
    DuplicateCount = 0
    GraphGCs = [CannonicalIndices(G, D, D.RootNodeName) for G in SaveGraphs]


#     GraphGCSet = set([CannonicalIndices(G, D, RootNodeName) for G in Tasks.keys()])
    Match = dict()
    for GC in GraphGCs:
        try:
            Match[len(GC)][tuple(GC[:2])].add(GC)
        except:
            try:
                Match[len(GC)][tuple(GC[:2])]=set([GC])
            except:
                Match[len(GC)] = dict()
                Match[len(GC)][tuple(GC[:2])]=set([GC])
    for i, G in enumerate(SaveGraphs):
#         GC = CannonicalIndices(G, D, RootNodeName)
#         GC = np.array(GC)
#         GC = tuple(GC)
        GC = GraphGCs[i]
        if GC in Match[len(GC)][GC[:2]]:
            Graphs[GC]=G
#             DuplicateCount += 1
#             try:
#                 InvalidGraphsCount[GC] += 1
#                 InvalidGraphs[GC].append(G)
#             except:
#                 InvalidGraphsCount[GC] = 2
#                 InvalidGraphs[GC] = []
#                 InvalidGraphs[GC].append(Graphs[GC])
#                 InvalidGraphs[GC].append(G)
        
    if Output:
        print "Duplicate Trees Found: {: >10}".format(len(GraphGCs)-len(Graphs))
        print "Unique Trees Found:    {: >10}".format(len(Graphs.keys()))
        print "-----------------------"+"-"*10
#     for x in InvalidGraphsCount.items():
#         print x
    return OrderedDict([x for x in sorted(Graphs.items(),reverse=True)])
#     return D, OrderedDict([x for x in sorted(Graphs.items(),reverse=True)]), OrderedDict([x for x in sorted(InvalidGraphs.items(),reverse=True)])

    #for k in Graphs.keys():
    #    print InList(k, SortedPathways.keys())
def ImportChunks(FileList):
    L = list()
    for File in FileList:
        with gzip.open(File, "r") as fh:
            L.extend(pickle.load(fh))
    return L
    

In [ ]:
%load_ext line_profiler

In [ ]:
%%time
D9, FullFileList9, GCFileList9 = AsmPathways(9, Output=True, SaveFiles=True)

Time Running:      4m12s | Chunk Time Delta:      4m12s | Graphs found so far:         500,000 | Current SaveGraphs Length:         500,000 | Current Memory Usage:     7.8GiB
File: FullPathways9_Chunk1.pickle written in 2m23s
File: PathwayGCs9_Chunk1.pickle written in   18s
Time Running:     12m36s | Chunk Time Delta:      5m40s | Graphs found so far:       1,000,000 | Current SaveGraphs Length:         500,000 | Current Memory Usage:    11.4GiB
File: FullPathways9_Chunk2.pickle written in 2m24s
File: PathwayGCs9_Chunk2.pickle written in   19s
Time Running:     21m16s | Chunk Time Delta:      5m55s | Graphs found so far:       1,500,000 | Current SaveGraphs Length:         500,000 | Current Memory Usage:    12.6GiB
File: FullPathways9_Chunk3.pickle written in 2m31s
File: PathwayGCs9_Chunk3.pickle written in   26s
Time Running:     29m35s | Chunk Time Delta:      5m20s | Graphs found so far:       2,000,000 | Current SaveGraphs Length:         500,000 | Current Memory Usage:    13.1GiB
